In [73]:
import openai
import numpy as np
import os
from dotenv import load_dotenv
import os
from collections import deque
from typing import Dict, List, Optional, Any
from geopy.geocoders import Nominatim
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union

from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
# Langchain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, LLMChain ,LLMCheckerChain
from langchain.callbacks import wandb_tracing_enabled
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    StringPromptTemplate
)
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from typing import Optional
from langchain.chains import SimpleSequentialChain ,SequentialChain
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent,BaseMultiActionAgent
from langchain.agents import AgentType, initialize_agent,AgentExecutor,BaseSingleActionAgent
from langchain.tools import tool
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser,Agent
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from langchain import OpenAI, SerpAPIWrapper

from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    AsyncCallbackManagerForToolRun,
    CallbackManagerForChainRun,
    CallbackManagerForToolRun,
    Callbacks,
)
from newspaper import Config, Article, Source
import requests
from utils.Webscraper import Webscraper
import tiktoken

from supabase import create_client, Client
import postgrest
from postgrest.exceptions import APIError
import newspaper

webscraper = Webscraper()

## Load Env Variables

In [75]:
# Load variables from the .env file
load_dotenv('./.env')

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT=os.getenv("LANGCHAIN_ENDPOINT")
# Access the variables
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Set the environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [51]:
# @tool("get_location",return_direct=False)
def get_coordinates(location:str) -> tuple[float, float]:
    """Returns the latitude and longitude of a location.Location should include any landmarks,cities, countries and addresses, output an address searchable in googleMaps be as specific as possible."""
    geolocator = Nominatim(user_agent="geoapi_explorer")
    try:
        location_info = geolocator.geocode(location)
    except:
        return "Location Input must be more general, E.g Country,City,state,street"
    
    if location_info:
        latitude = location_info.latitude
        longitude = location_info.longitude
        return (latitude, longitude)
    else:
        return "Location Input must be more general, E.g Country,City,state,street"


# use the function
coordinates = get_coordinates("Bali, Indonesia")
print(f'coordinates',coordinates)


coordinates (-8.3304977, 115.0906401)


# Steps for chains

### Disruption Event classifier 0|1 

In [52]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=1)
classifier_article_schema = {
    "name": "binary_classifier_article_schema",
    "description": "Binary Classifier Schema for Article, 0 for False and 1 for True",
    "type": "object",
    "properties": {
      "isDisruptionEvent": {
        "type": "boolean"
      },
      "Reason": {
        "type": "string"
      }
    },
    "required": ["isDisruptionEvent", "Reason"]
  }

classifierprompt = PromptTemplate(
    template = """Role:You are a Binary Classifier,your goal is to classify if the given news article is a vaild disruption event article or not.
    Conditions:
    1. A disruption event can be a Natural Disaster Example but not limited to: Earthquake,Flood,Extreme Weather. OR Man-Made but not limited to: Geo-Political,Protest/Riot,Airport Disruption.
    2. The disruption event the news article is reporting, must be a 'live' event, meaning it is currently happening. Not an article reporting on a past event.

    Article: {article}

    TASK: Given youre Role and the Conditions, Classify if the given news article is a vaild disruption event article or not. Think through and give reasoning for your decision. Must Output 0 for False and 1 for True.
    """,
    input_variables=["article"]
)
ArticleClassifier = create_structured_output_chain(output_schema=classifier_article_schema,llm = llm,prompt=classifierprompt)
ArticleClassifier

LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['article'], output_parser=None, partial_variables={}, template="Role:You are a Binary Classifier,your goal is to classify if the given news article is a vaild disruption event article or not.\n    Conditions:\n    1. A disruption event can be a Natural Disaster Example but not limited to: Earthquake,Flood,Extreme Weather. OR Man-Made but not limited to: Geo-Political,Protest/Riot,Airport Disruption.\n    2. The disruption event the news article is reporting, must be a 'live' event, meaning it is currently happening. Not an article reporting on a past event.\n\n    Article: {article}\n\n    TASK: Given youre Role and the Conditions, Classify if the given news article is a vaild disruption event article or not. Think through and give reasoning for your decision. Must Output 0 for False and 1 for True.\n    ", template_format='f-string', validate_tem

In [113]:
schema2 = ArticleClassifier.llm_kwargs
schema2

{'functions': [{'name': 'output_formatter',
   'description': 'Output formatter. Should always be used to format your response to the user.',
   'parameters': {'name': 'binary_classifier_article_schema',
    'description': 'Binary Classifier Schema for Article, 0 for False and 1 for True',
    'type': 'object',
    'properties': {'isDisruptionEvent': {'type': 'boolean'},
     'Reason': {'type': 'string'}},
    'required': ['isDisruptionEvent', 'Reason']}}],
 'function_call': {'name': 'output_formatter'}}

In [114]:
def extract_json_schema(data):
    # Check if the 'functions' key exists in the data
    if 'functions' in data:
        # Iterate through the functions
        for function in data['functions']:
            # Check if the 'parameters' key exists in the function
            if 'parameters' in function:
                # Check if the 'type' key is 'object' in the parameters
                if function['parameters'].get('type') == 'object':
                    # Return the properties and required fields of the object schema
                    properties = function['parameters'].get('properties', {})
                    required = function['parameters'].get('required', [])
                    return {'type': 'object', 'properties': properties, 'required': required}
    
    # If schema extraction fails, return None
    return None

# Example schema data
schema_data = {
    'functions': [
        {
            'name': 'output_formatter',
            'description': 'Output formatter. Should always be used to format your response to the user.',
            'parameters': {
                'name': 'binary_classifier_article_schema',
                'description': 'Binary Classifier Schema for Article, 0 for False and 1 for True',
                'type': 'object',
                'properties': {
                    'isDisruptionEvent': {'type': 'boolean'},
                    'Reason': {'type': 'string'}
                },
                'required': ['isDisruptionEvent', 'Reason']
            }
        }
    ],
    'function_call': {'name': 'output_formatter'}
}

# Extract the JSON schema
json_schema = extract_json_schema(schema2)

# Print the extracted JSON schema
print(json_schema)


{'type': 'object', 'properties': {'isDisruptionEvent': {'type': 'boolean'}, 'Reason': {'type': 'string'}}, 'required': ['isDisruptionEvent', 'Reason']}


In [117]:
import jsonschema
from jsonschema import ValidationError
# schema = {
#     "type": "object",
#     "properties": {
#         "isDisruptionEvent": {"type": "boolean"},
#         "Reason": {"type": "string"}
#     },
#     "required": ["isDisruptionEvent", "Reason"]
# }

sample_output = {
    "isDisruptionEvent": 'not a boolean testing',
    "Reason": "The article is reporting on a live event, a protest in Hong Kong."
}
# schema = ArticleClassifier.llm_kwargs

# Create a validator based on the JSON_SCHEMA
validator = jsonschema.Draft7Validator(json_schema)

# Check if the output is valid
errors = list(validator.iter_errors(sample_output))

if errors:
    error_messages = [str(error) for error in errors]
    print(False, ", ".join(error_messages))  # Output is invalid, print the validation error messages
else:
    print(True, "")  # Output is valid

False 'not a boolean testing' is not of type 'boolean'

Failed validating 'type' in schema['properties']['isDisruptionEvent']:
    {'type': 'boolean'}

On instance['isDisruptionEvent']:
    'not a boolean testing'


In [107]:
validator.check_schema(sample_output)

In [53]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
num_tokens_from_string("tiktoken is great!", "cl100k_base")


6

In [54]:
ArticleClassifier.llm_kwargs

{'functions': [{'name': 'output_formatter',
   'description': 'Output formatter. Should always be used to format your response to the user.',
   'parameters': {'name': 'binary_classifier_article_schema',
    'description': 'Binary Classifier Schema for Article, 0 for False and 1 for True',
    'type': 'object',
    'properties': {'isDisruptionEvent': {'type': 'boolean'},
     'Reason': {'type': 'string'}},
    'required': ['isDisruptionEvent', 'Reason']}}],
 'function_call': {'name': 'output_formatter'}}

In [55]:
locationExtractorSchema = {
    "name": "locationExtractorSchema",
    "description": "Format and extract the disruption event location from the given text",
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "Description": "Location of Disruption Event.Location should include any landmarks,cities, countries and addresses, output an address searchable in googleMaps be as specific as possible."
        },
        "Country": {
            "type": "string",
            "Description": "Country where the disruption event is happening"
        }
    },
    "required": ["location", "Country"]
}

locationExtractorPrompt = PromptTemplate(
    template = """Role:You are a Location Extractor,your goal is to extract the location of the disruption event from the given text. Location of Disruption Event. Examples: 1.French Pass, New Zealand 2.Xiamen Fujian Chain,3.Perry, Florida, USA. \n\nArticle:{article}\n\nTask: Extract Location of disruption event.Location should include any landmarks,cities, countries and addresses, output an address searchable in googleMaps be as specific as possible.Feedback:{feedback}""",
    input_variables=["article","feedback"]
)

locationExtractor = create_structured_output_chain(output_schema=locationExtractorSchema,llm = llm,prompt=locationExtractorPrompt)

In [56]:
eventDetails_schema = {
    "name": "eventDetails_schema",
    "description": "Format and extract the disruption event details from the given article",
    "type": "object",
    "properties": {
        "name": {
            "type": "string",
            "Description": "Name of Disruption Event, Includes some sort of key identifier"
        },
        "disruptionType": {
          "type": "string",
          "description": "Type of disruption event. Max 3 words"
        },
        "severity": {
          "type": "string",
          "description": "Quantifiable Severity metrics of the disruption event."
        },
        "date": {
            "type": "string",
            "description": "Date of Disruption Event, ISO 8601 format YYYY-MM-DD",
            "format": "date"
        },
        "radius": {
            "type": "number",
            "Description": "Estimated Radius of Disruption Event, in KM"
        }
    },
    "required": ["name", "disruptionType","severity","date","radius"]
}

eventDetailsPrompt = PromptTemplate(
    template = """Role:You are a Disruption event News Analyst, your goal is to extract the details of the disruption event from the given article. \n\nArticle:{article}\nFeedback:{feedback}\n\nTask: Extract the details of the disruption event from the given article.Details include:\n1.Name of Disruption Event, Includes some sort of key identifier,with indication of severity\n2.Type of disruption event ,Max 3 words.\n3.Quantifiable Severity metrics of the disruption event.Extract multiple metrics,E.g Casualities,Cost damage etc.Example Severity:"Magnitude of 5.6, Depth of 170km. tremor was felt widely across parts, but no damage was caused overall"\n4.Date of Disruption Event, in YYYY-MM-DD.\n5.Estimated Radius of imapact of Disruption Event, in KM.""",
    input_variables=["article","feedback"]
)

eventDetails = create_structured_output_chain(output_schema=eventDetails_schema,llm = llm,prompt=eventDetailsPrompt)

# Main input logic flow

In [82]:
import logging
logger = logging.getLogger(__name__)

class outputValidator:
    """Validates json output of any llmchain, given that it's created with the create_structured_output_chain function"""

    def _getOutputSchemaMapping(self, LLMChain:LLMChain) -> dict:
        """Returns the supposed output schema of the given LLMChain 
        
        Example llm_kwargs:
        {'functions': [{'name': 'output_formatter',
   'description': 'Output formatter. Should always be used to format your response to the user.',
   'parameters': {'name': 'binary_classifier_article_schema',
    'description': 'Binary Classifier Schema for Article, 0 for False and 1 for True',
    'type': 'object',
    'properties': {'isDisruptionEvent': {'type': 'boolean'},
     'Reason': {'type': 'string'}},
    'required': ['isDisruptionEvent', 'Reason']}}],
    'function_call': {'name': 'output_formatter'}}
    """
        output_schema = LLMChain.llm_kwargs
        # Create a dictionary mapping with Key as Key and Value as type
        output_schema = {key:value["type"] for key,value in output_schema.items()}
        return output_schema

        
    @classmethod
    def validate(cls, LLMChain:LLMChain,output:dict) -> Tuple[bool,str]:
        """Validates the output of the given llmchain
        Args:
            LLMChain (LLMChain): The LLMChain to validate
            output (dict): The output of the LLMChain
        Returns:
            Tuple[bool,str]: A tuple containing a boolean and a string, the boolean is True if the output is valid, False if not. 
            The string is the reason for the boolean value. If the boolean is True, the string will be empty.
            The string might be used to give feedback to LLMChain to improve the output.
        """
        
        # Check if the output is a dict
        if not isinstance(output,dict):
            return (False,"The output is not JSON object (dict)")
        
        #TODO: Check if the output is a valid json object from the schema
        return (True,"")

class MDE: 
    """Main Disruption Event Class, Handles the LLM chaining of input and outputs"""

    class MDE:
        binaryClassifier: LLMChain = ArticleClassifier
        locationExtractor: LLMChain = locationExtractor
        eventDetails: LLMChain = eventDetails 

        # The function to get the coordinates of a location
        get_coordinates: Callable = get_coordinates
        webscraper: Webscraper = webscraper

        article_token_threshold: int = 1500
        """The number of tokens to use the article summary instead of the full article"""

        @staticmethod
        def _articleExtraction(url) -> str:
            """Extracts the article text from the given url"""
            try:
                article = webscraper.scrape(url)
            except Exception as e:
                raise Exception(f"Error: Article Extraction Failed, {e}") from e
            return article

        @staticmethod
        def _binaryClassifier(article: Article) -> bool:
            """Classifies if the given article is a valid disruption event article or not"""
            try:
                # Get number of tokens for the article
                num_tokens = MDE.num_tokens_from_string(article.additional_data["text_body"], "cl100k_base")
                if num_tokens > MDE.article_token_threshold:
                    logger.info(f'Article Length: {num_tokens} tokens, using article summary instead')
                    result = MDE.binaryClassifier.run(article=article.summary)
                else:
                    result = MDE.binaryClassifier.run(article=article.additional_data["text_body"])
                # Validate the output
                validation_results = outputValidator.validate(MDE.binaryClassifier, result)
                if not validation_results[0]:
                    raise Exception(validation_results[1])

            except Exception as e:
                raise Exception(f"Error: Binary Classification Failed -> {e}") from e
        
            logger.info(f'Successfully classified isDisruptionEventarticle: {result["isDisruptionEvent"]}')
            return result["isDisruptionEvent"]
    
        @staticmethod
        def _locationExtractor(article: Article, feedback: str) -> str:
            """Extracts the location of the disruption event from the given article"""
            try:
                # Get number of tokens for the article
                num_tokens = MDE.num_tokens_from_string(article.additional_data["text_body"], "cl100k_base")
                if num_tokens > MDE.article_token_threshold:
                    logger.info(f'Article Length: {num_tokens} tokens, using article summary instead')
                    result = MDE.locationExtractor.run(article=article.summary, feedback=feedback)
                else:
                    result = MDE.locationExtractor.run(article=article.additional_data["text_body"], feedback=feedback)
                # Validate the output
                validation_results = outputValidator.validate(MDE.locationExtractor, result)
                if not validation_results[0]:
                    raise Exception(validation_results[1])

            except Exception as e:
                raise Exception(f"Error: Location Extraction Failed -> {e}") from e
        
            logger.info(f'Successfully extracted disruption location: {result["location"]}')
            return result["location"]

        @staticmethod
        def _eventDetails(article: Article, feedback: str) -> dict:
            """Extracts the event details of the disruption event from the given article"""
            try:
                # Get number of tokens for the article
                num_tokens = MDE.num_tokens_from_string(article.additional_data["text_body"], "cl100k_base")
                if num_tokens > MDE.article_token_threshold:
                    logger.info(f'Article Length: {num_tokens} tokens, using article summary instead')
                    result = MDE.eventDetails.run(article=article.summary, feedback=feedback)
                else:
                    result = MDE.eventDetails.run(article=article.additional_data["text_body"], feedback=feedback)
                # Validate the output
                validation_results = outputValidator.validate(MDE.eventDetails, result)
                if not validation_results[0]:
                    raise Exception(validation_results[1])

            except Exception as e:
                raise Exception(f"Error: Event Details Extraction Failed -> {e}") from e

            logger.info(f'Successfully extracted disruption Event Details: {result}')
            return result
    
        @staticmethod
        def articleAddParams(article: Article, params:dict) -> Article:
            """Adds the given params to Article.additional_data object"""
            # Add the params to the article dictionary
            article.additional_data.update(params)
            logger.info(f'Added number of params: {len(params)}')
            return article
    
        @staticmethod
        def num_tokens_from_string(string: str, encoding_name: str="cl100k_base") -> int:
            """Returns the number of tokens in a text string."""
            encoding = tiktoken.get_encoding(encoding_name)
            num_tokens = len(encoding.encode(string))
            return num_tokens

        @classmethod
        def _process(cls, article: Article, url: str = None) -> Article:
            """Given an article or a url, use the LLMChains to extract the disruption event information"""
            try:
                if url:
                    # Extract the article from the url as a newspaper3k Article object
                    article = cls._articleExtraction(url)

                # Check if the article is a disruption event article
                if not cls._binaryClassifier(article):
                    raise Exception("Error: Article is not a disruption event article")
            
                # Extract the location of the disruption event
                location = cls._locationExtractor(article, feedback="")
                logger.info(f'Location extracted: {location}')
                coordinates_info = cls.get_coordinates(location)
                logger.info(f'Coordinates info: {coordinates_info}')
                # Loop until the coordinates are valid
                max_retries = 3
                retries = 0
                while isinstance(coordinates, str) and retries < max_retries:
                    logger.info(f'Location is not valid, please try again. Location: {location} coordinates: {coordinates_info}')
                    location = cls._locationExtractor(article, feedback=coordinates_info)
                    coordinates_info = cls.get_coordinates(location)
                    retries += 1

                # Extract the disruption event information
                event_details = cls._eventDetails(article, feedback="")

                article = cls.articleAddParams(article, {"location": location, "coordinates": coordinates_info})
                article = cls.articleAddParams(article, event_details)
                return article
        
            except Exception as e:
                logger.error(e)
                return str(e)

        @classmethod
        def processUrl(cls, url: str) -> Article:
            """Given a url, use the LLMChains to extract the disruption event information"""
            return cls._process(None, url)

        @classmethod
        def process(cls, article: Article) -> Article:
            """Main processing function, given an article, use the LLMChains to extract the disruption event information"""
            return cls._process(article)

# Testing the code

In [83]:
# Process a URL to extract disruption event information
final_article = MDE.processUrl(url = "https://www.usnews.com/news/us/articles/2023-08-30/idalia-predicted-to-hit-florida-as-category-4-hurricane-with-catastrophic-storm-surge")

# Print the extracted information
print(final_article)

2023-09-11 15:09:53,590:INFO - Article Length: 1710 tokens, using article summary instead
2023-09-11 15:09:56,842:INFO - Successfully classified isDisruptionEventarticle: True
2023-09-11 15:09:56,860:INFO - Article Length: 1710 tokens, using article summary instead
2023-09-11 15:09:58,159:INFO - Successfully extracted disruption location: Keaton Beach, Florida, USA
2023-09-11 15:09:58,164:INFO - Location extracted: Keaton Beach, Florida, USA
2023-09-11 15:09:59,037:INFO - Coordinates info: (29.8252983, -83.5938806)
2023-09-11 15:09:59,041:INFO - Article Length: 1710 tokens, using article summary instead
2023-09-11 15:10:02,552:INFO - Successfully extracted disruption Event Details: {'name': 'Hurricane Idalia', 'disruptionType': 'Hurricane', 'severity': 'Category 3', 'date': 'YYYY-MM-DD', 'radius': 0}
2023-09-11 15:10:02,554:INFO - Added number of params: 2
2023-09-11 15:10:02,556:INFO - Added number of params: 5


# Testing Supabase 

In [70]:
class DataInsertor:
    """
    TLDR: "Fancy INSERT INTO statement for the supabase" (Need to migrate over)

    Methods:
        * addDisruptionEvent(self, DisruptionEvent: dict)
        * addEventDataRelation
    """

    def __init__(self, supabase: Client):
        self.supabase = supabase


    def addArticleData(self, article:Article):
        article_dict = self._formatArticleDict(article)
        try:
            data, count = self._insertArticleData(article_dict)
            id = self._extractArticleId(data)
            article_dict['id'] = id
            return article_dict
        except postgrest.exceptions.APIError as e:
            self._handleInsertError(e, article_dict)

    def _formatArticleDict(self, article:Article):
        return {
            "Title": article.title,
            "Text": article.text,
            "Location": article.additional_data["location"],
            "lat": article.additional_data["coordinates"][0],
            "lng": article.additional_data["coordinates"][1],
            "DisruptionType": article.additional_data["disruptionType"],
            "Severity": article.additional_data["severity"],
            "SourceName": article.source_url,
            "Url": article.url,
            "ImageUrl": article.top_image,
            # "PublishedDate": article.publish_date,
            "Radius": article.additional_data["radius"]*1000, # Convert to meters
        }

    def _insertArticleData(self, article_dict:dict):
        return supabase.table("Article").insert(article_dict).execute()

    def _extractArticleId(self, data):
        return data[1][0]['id']

    def _handleInsertError(self, error, article_dict:dict):
        print(f'Error inserting into Supabase: {error}')
        print(error.code, error.message, error.details)
        if error.code == '23505':
            print(f'Article with URL {article_dict["Url"]} already exists in the database.')
            existing_data = supabase.table(self.table).select('*').eq('Url', article_dict['Url']).execute()
            return existing_data.data[0]
        else:
            print(f'Unexpected error: {error}')